In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

import numpy as np
import pandas as pd

spark = SparkSession.builder.getOrCreate()

## Data Acquisition

*This exercises uses the case.csv, dept.csv, and source.csv files from the san antonio 311 call dataset.*

### Part I

**1. Read the case, department, and source data into their own spark dataframes.**

In [4]:
case = spark.read.csv('case.csv', 
                       sep=',', 
                       header=True, 
                       inferSchema=True)

dept = spark.read.csv('dept.csv', 
                       sep=',', 
                       header=True, 
                       inferSchema=True)

source = spark.read.csv('source.csv', 
                         sep=',', 
                         header=True,
                         inferSchema=True)

**2. Let's see how writing to the local disk works in spark:**

- Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json
- Inspect your folder structure. What do you notice?

In [6]:
#write to json
source.write.json('source_json')
#write to csv
source.write.csv('source_csv')

*After inspecting,I notice that source_json and source_csv where stored as directories and not as files.*

**3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.**

In [10]:
# start with case
case.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'int')]

In [13]:
case.show(3, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 1/1/18 0:46          
 case_closed_date     | 1/3/18 8:11          
 SLA_due_date         | 1/5/18 8:30          
 case_late            | NO                   
 num_days_late        | -2.0126041

In [16]:
# I need to change several data types including converting the date time
fmt = 'M/d/yy H:mm'

case = (
    case.withColumn('case_id', case.case_id.cast('string'))
        .withColumn('case_opened_date', to_timestamp('case_opened_date', fmt))
        .withColumn('case_closed_date', to_timestamp('case_closed_date', fmt))
        .withColumnRenamed('SLA_due_date', 'case_due_date')
        .withColumn('case_due_date', to_timestamp('case_due_date', fmt))
        .withColumn('case_closed', expr('case_closed == "YES"'))
        .withColumn('case_late', expr('case_late == "YES"'))
        .withColumn('council_district', case.council_district.cast('string'))
        .withColumn('case_age', datediff(current_timestamp(), 'case_opened_date'))
        .withColumn('days_to_closed', datediff('case_closed_date', 'case_opened_date'))
        .withColumn('case_lifetime', when(expr('! case_closed'), col('case_age'))
                                      .otherwise(col('days_to_closed')))
)  
case.dtypes

[('case_id', 'string'),
 ('case_opened_date', 'timestamp'),
 ('case_closed_date', 'timestamp'),
 ('case_due_date', 'timestamp'),
 ('case_late', 'boolean'),
 ('num_days_late', 'double'),
 ('case_closed', 'boolean'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'string'),
 ('case_age', 'int'),
 ('days_to_closed', 'int'),
 ('case_lifetime', 'int')]

In [17]:
# next source
source.dtypes

[('source_id', 'string'), ('source_username', 'string')]

In [18]:
source.show(5) 

+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
|   103582|     Carmen Cura|
|   106463| Richard Sanchez|
|   119403|  Betty De Hoyos|
|   119555|  Socorro Quiara|
+---------+----------------+
only showing top 5 rows



*This seems like the appropriate dtypes*

In [19]:
# Last, dept
dept.dtypes

[('dept_division', 'string'),
 ('dept_name', 'string'),
 ('standardized_dept_name', 'string'),
 ('dept_subject_to_SLA', 'string')]

In [20]:
dept.show(5)

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|                YES|
|               Brush|Solid Waste Manag...|           Solid Waste|                YES|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|                YES|
|    Code Enforcement|Code Enforcement ...|  DSD/Code Enforcement|                YES|
+--------------------+--------------------+----------------------+-------------------+
only showing top 5 rows



*All seem like appropriate data types but I'll change dept_subject_to_SLA to a boolean value instead of YES, NO.*

In [21]:
# converting string to boolean value
dept = dept.withColumn('dept_subject_to_SLA', expr('dept_subject_to_SLA == "YES"'))

dept.show(5)

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|               true|
|               Brush|Solid Waste Manag...|           Solid Waste|               true|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|               true|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|               true|
|    Code Enforcement|Code Enforcement ...|  DSD/Code Enforcement|               true|
+--------------------+--------------------+----------------------+-------------------+
only showing top 5 rows



### Part II

**1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?**

In [24]:
case.select(max('case_age')).show()

+-------------+
|max(case_age)|
+-------------+
|         1968|
+-------------+



**2. How many Stray Animal cases are there?**

In [25]:
case.filter(col('service_request_type') == 'Stray Animal').count()

26760

**3. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?**

In [26]:
field_operations = case.filter(col('dept_division') == 'Field Operations')
field_operations.filter(col('service_request_type') != 'Officer Standby').count()

113902

**4. Convert the council_district column to a string column.**

In [27]:
case = case.withColumn('council_district', col('council_district').cast('string'))

In [28]:
#sanity check
case.dtypes

[('case_id', 'string'),
 ('case_opened_date', 'timestamp'),
 ('case_closed_date', 'timestamp'),
 ('case_due_date', 'timestamp'),
 ('case_late', 'boolean'),
 ('num_days_late', 'double'),
 ('case_closed', 'boolean'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'string'),
 ('case_age', 'int'),
 ('days_to_closed', 'int'),
 ('case_lifetime', 'int')]

**5. Extract the year from the case_closed_date column.**

In [29]:
case = case.withColumn("case_closed_year", year("case_closed_date"))

case.show(2, False, True)

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 2018-01-01 00:42:00                  
 case_closed_date     | 2018-01-01 12:29:00                  
 case_due_date        | 2020-09-26 00:42:00                  
 case_late            | false                                
 num_days_late        | -998.5087616000001                   
 case_closed          | true                                 
 dept_division        | Field Operations                     
 service_request_type | Stray Animal                         
 SLA_days             | 999.0                                
 case_status          | Closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  EL PASO ST, San Antonio, 78207 
 council_district     | 5                                    
 case_age             | 1603                                 
 days_to

**6. Convert num_days_late from days to hours in new columns num_hours_late.**

In [30]:
case = case.withColumn('num_hours_late', expr('num_days_late*24'))

case.select("num_days_late", "num_hours_late").show(5)

+-------------------+-------------------+
|      num_days_late|     num_hours_late|
+-------------------+-------------------+
| -998.5087616000001|     -23964.2102784|
|-2.0126041669999997|-48.302500007999996|
|       -3.022337963|      -72.536111112|
|       -15.01148148|      -360.27555552|
|0.37216435200000003|  8.931944448000001|
+-------------------+-------------------+
only showing top 5 rows



**7. Join the case data with the source and department data.**

In [35]:
df = (
    case.join(source, on='source_id', how='left')
        .join(dept, on='dept_division', how='left')
        .drop('dept_name')
        .withColumnRenamed('standardized_dept_name', 'department')
        .withColumn('dept_subject_to_SLA', (col('dept_subject_to_SLA') == 'true'))
)

In [36]:
df.show(1, vertical=True)

-RECORD 0------------------------------------
 dept_division        | Field Operations     
 source_id            | svcCRMLS             
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
 case_age             | 1603                 
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 case_closed_year     | 2018                 
 num_hours_late       | -23964.2102784       
 source_username      | svcCRMLS             
 department           | Animal Car

**8. Are there any cases that do not have a request source?**

In [39]:
df.filter(col('source_id').isNull() | isnan('source_id') | (col('source_id') == '')).count()

0

**9. What are the top 10 service request types in terms of number of requests?**

In [40]:
(df.groupby('service_request_type')
      .count()
      .sort(col('count').desc())
      .show(10, truncate=False)
)

+--------------------------------+-----+
|service_request_type            |count|
+--------------------------------+-----+
|No Pickup                       |89210|
|Overgrown Yard/Trash            |66403|
|Bandit Signs                    |32968|
|Damaged Cart                    |31163|
|Front Or Side Yard Parking      |28920|
|Stray Animal                    |27361|
|Aggressive Animal(Non-Critical) |25492|
|Cart Exchange Request           |22608|
|Junk Vehicle On Private Property|21649|
|Pot Hole Repair                 |20827|
+--------------------------------+-----+
only showing top 10 rows



**10. What are the top 10 service request types in terms of average days late?**

In [41]:
(df.groupby('service_request_type')
      .agg(mean('num_days_late'))
      .sort(col('avg(num_days_late)').desc())
      .show(10, truncate=False))

+--------------------------------------+------------------+
|service_request_type                  |avg(num_days_late)|
+--------------------------------------+------------------+
|Zoning: Junk Yards                    |175.95636210420943|
|Labeling for Used Mattress            |162.43032902285717|
|Record Keeping of Used Mattresses     |153.99724039428568|
|Signage Requied for Sale of Used Mattr|151.63868055333333|
|Storage of Used Mattress              |142.11255641500003|
|Zoning: Recycle Yard                  |135.92851612479797|
|Donation Container Enforcement        |131.75610506358706|
|License Requied Used Mattress Sales   |128.79828704142858|
|Traffic Signal Graffiti               |101.79846062200002|
|Complaint                             |72.8705023031169  |
+--------------------------------------+------------------+
only showing top 10 rows



**11. Does number of days late depend on department?**

In [43]:
(df.groupby('department')
      .agg(mean('num_days_late'))
      .sort(col('avg(num_days_late)').desc())
      .show())

+--------------------+-------------------+
|          department| avg(num_days_late)|
+--------------------+-------------------+
|    Customer Service|  59.73709149630077|
|         Solid Waste| -2.200057513672164|
|        Metro Health| -4.911766979607004|
|  Parks & Recreation| -5.251521960055145|
|Trans & Cap Impro...|-20.612837354052708|
|DSD/Code Enforcement| -38.36938892614506|
|Animal Care Services|-226.51783940550334|
|        City Council|               null|
+--------------------+-------------------+



**12. How do number of days late depend on department and request type?**

In [44]:
(df.groupby('department', 'service_request_type')
      .agg(mean('num_days_late'))
      .sort('department', 'service_request_type')
      .show(truncate=False))


+--------------------+--------------------------------------+--------------------+
|department          |service_request_type                  |avg(num_days_late)  |
+--------------------+--------------------------------------+--------------------+
|Animal Care Services|Aggressive Animal(Critical)           |16.69636881189224   |
|Animal Care Services|Aggressive Animal(Non-Critical)       |2.645033883163268   |
|Animal Care Services|Animal Bite(Critical)                 |0.019096834813110173|
|Animal Care Services|Animal Bite(Non-Critical)             |-2.505667547973447  |
|Animal Care Services|Animal Cruelty(Critical)              |-0.06219277353522766|
|Animal Care Services|Animal Neglect                        |7.332927878069383   |
|Animal Care Services|Animal Permits Request                |22.19792271447681   |
|Animal Care Services|City Council Animal Request           |-1.5475013634821913 |
|Animal Care Services|Injured Animal(Critical)              |-0.06529248474639532|
|Ani